In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from ray.tune.schedulers import PopulationBasedTraining
from sklearn.model_selection import KFold
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score

In [6]:
df=pd.read_csv('./cleaned_datasets/bike_data.csv',index_col=0)  

In [7]:
df

,yr,temp,hum,windspeed,cnt,days_since_2011,season_FALL,season_SPRING,season_SUMMER,season_WINTER,holiday_HOLIDAY,workingday_WORKING DAY,weathersit_GOOD,weathersit_MISTY,weathersit_RAIN/SNOW/STORM
0,2011,8.175849,80.5833,10.749882,985,0,0,0,0,1,0,0,0,1,0
1,2011,9.083466,69.6087,16.652113,801,1,0,0,0,1,0,0,0,1,0
2,2011,1.229108,43.7273,16.636703,1349,2,0,0,0,1,0,1,1,0,0
3,2011,1.400000,59.0435,10.739832,1562,3,0,0,0,1,0,1,1,0,0
4,2011,2.666979,43.6957,12.522300,1600,4,0,0,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2012,3.945849,65.2917,23.458911,2114,726,0,0,0,1,0,1,0,1,0
727,2012,3.906651,59.0000,10.416557,3095,727,0,0,0,1,0,1,0,1,0
728,2012,3.906651,75.2917,8.333661,1341,728,0,0,0,1,0,0,0,1,0
729,2012,4.024151,48.3333,23.500518,1796,729,0,0,0,1,0,0,1,0,0


In [8]:
y = df.pop('cnt')
X = df
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [9]:
X_train

,yr,temp,hum,windspeed,days_since_2011,season_FALL,season_SPRING,season_SUMMER,season_WINTER,holiday_HOLIDAY,workingday_WORKING DAY,weathersit_GOOD,weathersit_MISTY,weathersit_RAIN/SNOW/STORM
202,2011,31.871651,58.0417,8.917700,202,0,0,1,0,0,1,1,0,0
402,2012,8.645849,49.6250,9.874393,402,0,0,0,1,0,1,1,0,0
11,2011,0.118169,59.9545,20.410009,11,0,0,0,1,0,1,1,0,0
307,2011,10.956651,62.2500,18.209193,307,1,0,0,0,0,1,0,1,0
688,2012,9.899151,62.3333,15.749489,688,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2011,22.510849,73.5833,9.582943,167,0,1,0,0,0,1,1,0,0
10,2011,-0.052723,68.6364,8.182844,10,0,0,0,1,0,1,0,1,0
403,2012,4.063349,72.2917,8.959307,403,0,0,0,1,0,1,0,1,0
495,2012,15.774151,55.2083,21.042221,495,0,1,0,0,0,1,1,0,0


In [10]:
y_train

202    3387
402    4375
11     1162
307    4046
688    5499
       ... 
167    4844
10     1263
403    2802
495    6572
118    4595
Name: cnt, Length: 584, dtype: int64

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
class DataFrameDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = torch.tensor(X_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [13]:
def create_dataloader(X_train, X_test, y_train, y_test, batch_size=32):
    # Create custom Dataset objects for train and test data
    train_dataset = DataFrameDataset(X_train, y_train)
    test_dataset = DataFrameDataset(X_test, y_test)

    # Create DataLoaders for train and test data
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [14]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, hidden_layers, hidden_units, dropout, dropout_array, input_dim, output_dim, activation=nn.ReLU(),norm=False):
        if hidden_layers != len(hidden_units):
            print("Error: wrong size of hidden_layers or hidden_units")
            return
        layers = []
        i = 0
        if norm:
            layers.append(nn.BatchNorm1d(input_dim))

        super(Net, self).__init__()
        
        layers.append(nn.Linear(input_dim, hidden_units[i]))
        layers.append(activation)
        if dropout:
            layers.append(nn.Dropout(dropout_array[i]))

        for _ in range(hidden_layers - 1):
            i += 1
            layers.append(nn.Linear(hidden_units[i-1], hidden_units[i]))
            layers.append(activation)
            if dropout:
                layers.append(nn.Dropout(dropout_array[i]))

        layers.append(nn.Linear(hidden_units[-1], output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)


In [15]:
model = Net(hidden_layers=3, hidden_units=[24, 24, 24], dropout=True, dropout_array=[0.2, 0.3, 0.4], input_dim=13, output_dim=1, activation=nn.Sigmoid(),norm=True)


In [16]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = model.to(device)

In [17]:
summary(model,input_size=(13,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                   [-1, 13]              26
            Linear-2                   [-1, 24]             336
           Sigmoid-3                   [-1, 24]               0
           Dropout-4                   [-1, 24]               0
            Linear-5                   [-1, 24]             600
           Sigmoid-6                   [-1, 24]               0
           Dropout-7                   [-1, 24]               0
            Linear-8                   [-1, 24]             600
           Sigmoid-9                   [-1, 24]               0
          Dropout-10                   [-1, 24]               0
           Linear-11                    [-1, 1]              25
Total params: 1,587
Trainable params: 1,587
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [18]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [19]:
def train(config, checkpoint_dir=None):
    hidden_layers=config["hidden_layers"]
    hidden_units=config["hidden_units"]
    dropout=config["dropout"]
    dropout_array=config["dropout_array"]
    activation=config["activation"]
    norm=config["norm"]
    lr=config["lr"]
    input_dim=X_train.shape[1]
    output_dim=1
    
    train_ds = TensorDataset(torch.Tensor(X_train_scaled), torch.Tensor(y_train.to_numpy()))
    test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.to_numpy()))
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold = 0
    avg_loss = 0
    avg_r2 = 0

    for train_idx, val_idx in kfold.split(train_ds):
        print("This is fold :" ,fold)
        #trainloader, testloader = create_dataloader(X_train_scaled, X_test_scaled, y_train.values, y_test.values, batch_size=config["batch_size"])
        net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=input_dim, output_dim=output_dim, activation=activation,norm=norm)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)
        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])
        train_subset = torch.utils.data.Subset(train_ds, train_idx)
        val_subset = torch.utils.data.Subset(train_ds, val_idx)

        train_loader = DataLoader(train_subset, batch_size=config["batch_size"], shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=config["batch_size"])
    
        for epoch in range(config["epochs"]):
            running_loss = 0.0
            train_loss=0.0
            val_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            train_loss=running_loss/len(train_loader)
               
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()      
            avg_loss += val_loss / len(val_loader)
            y_true = y_test.to_numpy()
            y_pred = net(torch.Tensor(X_test_scaled).to(device)).cpu()
            r2 = r2_score(y_true, y_pred)
            print(r2)
            tune.report(r2_curr=r2)
            avg_r2 += r2
        fold += 1
        

    avg_loss /= fold
    avg_r2 /= fold
    print("Average loss across folds: ", avg_loss)
    print("Average R2 score across folds: ", avg_r2)
    tune.report(loss=avg_loss,train_loss=train_loss,r2_score=avg_r2,r2_curr=r2)

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
device

'cuda'

In [22]:
def generate_hidden_units(hidden_layers):
    return np.random.choice(a=[16,32,64,128], size=hidden_layers).tolist()

search_space = {
    "batch_size": tune.choice([8,16,32, 64, 128]),
    "lr": tune.loguniform(1e-7, 1e-2),
    "epochs": tune.choice([25,50, 100, 150,300,500]),
    #"epochs": tune.choice([25]),
    "hidden_layers": tune.randint(1, 4),
    "hidden_units": tune.sample_from(lambda spec: generate_hidden_units(spec.config.hidden_layers)),
    "dropout": tune.choice([True, False]),
    "dropout_array": tune.sample_from(lambda spec: np.random.uniform(low=0.1, high=0.8, size=spec.config.hidden_layers).tolist()),
    "activation": tune.choice([nn.ReLU()]),
    "norm": tune.choice([True, False])
}

In [19]:
scheduler = ASHAScheduler(metric="r2_curr", mode="max", grace_period=2, reduction_factor=2)
"""
pbt_scheduler = PopulationBasedTraining(
    time_attr='training_iteration',
    metric='loss',
    mode='min',
    perturbation_interval=1,
    hyperparam_mutations=search_space
)
"""
reporter = CLIReporter(metric_columns=["loss", "training_iteration","train_loss"])

In [96]:
analysis = tune.run(
        train,
        resources_per_trial={"cpu": 8, "gpu": 1 if torch.cuda.is_available() else 0},
        config=search_space,
        num_samples=100,
        scheduler=scheduler,
        progress_reporter=reporter)

print("Best hyperparameters found were: ", analysis.get_best_config(metric="r2_score", mode="max"))


== Status ==
Current time: 2023-04-10 14:38:07 (running for 00:00:00.22)
Memory usage on this node: 8.4/15.9 GiB 
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: 0.154901297861894 | Iter 2.000: -1.924830585977822
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/4.26 GiB heap, 0.0/2.13 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-10_14-38-07
Number of trials: 17/100 (16 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |
|-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+---------

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,r2_curr,r2_score,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,training_iteration,trial_id,warmup_time
train_8ba8c_00000,2023-04-10_14-38-18,True,,1787c9a7eeab4613ba457f86a868ff29,,DESKTOP-8SILQJU,2,,127.0.0.1,28976,-4.84671,,6.97117,2.65341,6.97117,1681130298,0,,,2,8ba8c_00000,0.00400496
train_8ba8c_00001,2023-04-10_14-38-37,True,,51b15d963bfa42f19a578c9971b4ee62,,DESKTOP-8SILQJU,2,,127.0.0.1,21292,-4.84632,,15.106,6.84222,15.106,1681130317,0,,,2,8ba8c_00001,0.00400352
train_8ba8c_00002,2023-04-10_14-38-44,True,,20003fdff1f9455798a47ffae496e028,,DESKTOP-8SILQJU,2,,127.0.0.1,18568,-4.84723,,2.86552,0.599545,2.86552,1681130324,0,,,2,8ba8c_00002,0.00400305
train_8ba8c_00003,2023-04-10_14-38-53,True,,5f539b9b611c4fe691e964921cc927ac,,DESKTOP-8SILQJU,4,,127.0.0.1,27676,-4.83728,,4.64647,0.787716,4.64647,1681130333,0,,,4,8ba8c_00003,0.004004
train_8ba8c_00004,2023-04-10_14-39-08,True,,3dc921dc4b444ad59b505b5f93899fea,,DESKTOP-8SILQJU,4,,127.0.0.1,14896,-4.7787,,10.0261,2.06588,10.0261,1681130348,0,,,4,8ba8c_00004,0.00400352
train_8ba8c_00005,2023-04-10_14-39-18,True,,df04aa66981f4fa3a9ce2a90022ccd00,,DESKTOP-8SILQJU,2,,127.0.0.1,7408,-4.84594,,6.71411,2.52129,6.71411,1681130358,0,,,2,8ba8c_00005,0.00400305
train_8ba8c_00006,2023-04-10_14-39-25,True,,8f7ed14382df45b68ea512c50b2c5da7,"6_activation=ReLU,batch_size=128,dropout=False,dropout_array=0_18600201757706572_0_17164307794715505_0_2020983558822435,epochs=25,hidden_layers=3,hidden_units=128_16_128,lr=0.0049,norm=False",DESKTOP-8SILQJU,6,1010.7028564453125,127.0.0.1,13168,0.797003,0.7854788273586923,2.81015,0.00300288,2.81015,1681130365,0,,949.4258575439453,6,8ba8c_00006,0.00400329
train_8ba8c_00007,2023-04-10_14-40-46,True,,d91d0c2e408f42bd99bc5f17d99ebe14,"7_activation=ReLU,batch_size=16,dropout=True,dropout_array=0_34502799147000773_0_2657254777146101_0_682046827084674,epochs=300,hidden_layers=3,hidden_units=128_16_16,lr=0.0009,norm=False",DESKTOP-8SILQJU,6,2078.462710571289,127.0.0.1,13108,0.0865635,-0.07727292222938409,76.7618,0.00700712,76.7618,1681130446,0,,2211.814762369792,6,8ba8c_00007,0.004004
train_8ba8c_00008,2023-04-10_14-40-57,True,,5e731f626b834418b3fc1047a70bfbb7,,DESKTOP-8SILQJU,4,,127.0.0.1,22632,-3.82438,,6.65305,1.19008,6.65305,1681130457,0,,,4,8ba8c_00008,0.00400448
train_8ba8c_00009,2023-04-10_14-41-20,True,,3ccc53864afb486e8a0f6d70f37bcfee,,DESKTOP-8SILQJU,2,,127.0.0.1,852,-4.84719,,19.8785,9.03722,19.8785,1681130480,0,,,2,8ba8c_00009,0.004004


== Status ==
Current time: 2023-04-10 14:38:15 (running for 00:00:07.93)
Memory usage on this node: 10.6/15.9 GiB 
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: 0.154901297861894 | Iter 2.000: -1.924830585977822
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/4.26 GiB heap, 0.0/2.13 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-10_14-38-07
Number of trials: 18/100 (17 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------------------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |   training_iteration |
|-------------------+----------+-----------------+--------------+------------

2023-04-10 14:38:18,215	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}


(train pid=28976) -4.846710630467826
(train pid=21292) This is fold : 0
== Status ==
Current time: 2023-04-10 14:38:24 (running for 00:00:16.65)
Memory usage on this node: 10.5/15.9 GiB 
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: 0.154901297861894 | Iter 2.000: -3.6584179548451052
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/4.26 GiB heap, 0.0/2.13 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-10_14-38-07
Number of trials: 19/100 (17 PENDING, 1 RUNNING, 1 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |   trainin

...

2023-04-10 15:20:46,862	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}


== Status ==
Current time: 2023-04-10 15:20:52 (running for 00:42:45.19)
Memory usage on this node: 9.6/15.9 GiB 
Using AsyncHyperBand: num_stopped=81
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -0.309347095701594 | Iter 2.000: -4.84615766800033
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/4.26 GiB heap, 0.0/2.13 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-10_14-38-07
Number of trials: 100/100 (1 RUNNING, 99 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |     loss |   training_iteration |   train_loss |
|----------------

2023-04-10 15:20:54,856	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}
2023-04-10 15:20:55,020	INFO tune.py:762 -- Total run time: 2567.40 seconds (2567.23 seconds for the tuning loop).


== Status ==
Current time: 2023-04-10 15:20:54 (running for 00:42:47.25)
Memory usage on this node: 9.8/15.9 GiB 
Using AsyncHyperBand: num_stopped=81
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -0.2794312989194925 | Iter 2.000: -4.846047279164649
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/4.26 GiB heap, 0.0/2.13 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-10_14-38-07
Number of trials: 100/100 (100 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |     loss |   training_iteration |   train_loss |
|-------------------+--------

Best hyperparameters found were:  {'batch_size': 8, 'lr': 0.00035262507645898283, 'epochs': 500, 'hidden_layers': 3, 'hidden_units': [128, 128, 64], 'dropout': False, 'dropout_array': [0.39233452918726697, 0.3225943639867505, 0.6472032609256847], 'activation': ReLU(), 'norm': False}


In [109]:
analysis.results_df.sort_values(by=['r2_score', 'loss'],na_position='first').tail(10)

,r2_curr,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,config/epochs,config/hidden_layers,config/hidden_units,config/dropout,config/dropout_array,config/activation,config/norm,loss,train_loss,r2_score
trial_id,,,,,,,,,,,,,,,,,,,,,
8ba8c_00006,0.797003,0.003003,True,None,None,6,8f7ed14382df45b68ea512c50b2c5da7,2023-04-10_14-39-25,1681130365,2.810148,...,25,3,"[128, 16, 128]",False,"[0.18600201757706572, 0.17164307794715505, 0.2...",ReLU(),False,1010.702856,949.425858,0.785479
8ba8c_00077,0.795164,0.020017,True,None,None,6,db220c6247eb4e8ebeb225fcf3b08a15,2023-04-10_15-07-46,1681132066,77.997461,...,300,2,"[128, 32]",False,"[0.3922780611275247, 0.753126536377794]",ReLU(),True,955.687418,940.824581,0.785762
8ba8c_00082,0.802703,0.002000,True,None,None,6,a58ab7915c9d4383803a2439bb956089,2023-04-10_15-09-50,1681132190,7.822773,...,100,1,[16],False,[0.5137332568496699],ReLU(),True,897.789783,857.877754,0.786664
8ba8c_00048,0.784524,0.015014,True,None,None,6,d59df167e7ed4fdd8713fe511ebd36c4,2023-04-10_14-56-49,1681131409,44.052242,...,150,3,"[16, 16, 64]",False,"[0.19320951365603362, 0.7498518441142713, 0.79...",ReLU(),True,943.535123,825.144670,0.792701
8ba8c_00099,0.806874,0.003003,True,None,None,6,ebf8189d6451470fb4a90128f757ca37,2023-04-10_15-20-54,1681132854,3.596169,...,25,3,"[16, 64, 16]",False,"[0.5828298866319033, 0.7033176851746625, 0.364...",ReLU(),False,930.813696,917.532372,0.795279
8ba8c_00015,0.813505,0.008006,True,None,None,6,3ac01fabc7c3411caf25ca35387470b1,2023-04-10_14-43-30,1681130610,9.767884,...,100,3,"[16, 64, 64]",False,"[0.562357872242018, 0.5706455582835319, 0.1130...",ReLU(),True,878.038690,844.793564,0.802540
8ba8c_00083,0.844919,0.021020,True,None,None,6,2c2d5cfbf6ab44d2b013f2093170c897,2023-04-10_15-10-28,1681132228,33.642033,...,150,2,"[64, 16]",False,"[0.41333905094304635, 0.3525725313233301]",ReLU(),False,799.424117,707.131231,0.844818
8ba8c_00069,0.861852,0.019017,True,None,None,6,ef9778d111f449bda2e8d33c4ca55d02,2023-04-10_15-05-17,1681131917,29.120877,...,500,1,[128],False,[0.7458238807241184],ReLU(),False,805.275415,647.954926,0.853829
8ba8c_00036,0.884175,0.012017,True,None,None,6,71cb00e4b3d148faabcded946f5c8b98,2023-04-10_14-51-27,1681131087,62.069462,...,300,2,"[32, 64]",False,"[0.767973236566046, 0.21074933115186717]",ReLU(),False,725.903327,456.210255,0.873673


In [37]:
x=np.linspace(400,10000,101) 
plt.scatter(y_test_pred.detach().numpy(), y_test_con)
plt.plot(x,x,'r')
plt.show()

NameError: name 'y_test_pred' is not defined

In [20]:
cfg = {'batch_size': 128, 'lr': 0.0033598292706856687, 'epochs': 50, 'hidden_layers': 3, 'hidden_units': [128, 16, 16], 'dropout': False, 'dropout_array': [0.6381608360342922, 0.4302850889146931, 0.7460575980141309], 'activation': nn.ReLU(), 'norm': True}
hidden_layers=cfg["hidden_layers"]
hidden_units=cfg["hidden_units"]
dropout=cfg["dropout"]
dropout_array=cfg["dropout_array"]
activation=cfg["activation"]
norm=cfg["norm"]
lr=cfg["lr"]

In [21]:
net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=14, output_dim=1, activation=activation,norm=norm)

In [ ]:
analysis.get_best_checkpoint

In [ ]:
net.load_state_dict(analysis.get_best_checkpoint)

In [114]:
best_config={'batch_size': 8, 'lr': 0.00035262507645898283, 'epochs': 500, 'hidden_layers': 3, 'hidden_units': [128, 128, 64], 'dropout': False, 'dropout_array': [0.39233452918726697, 0.3225943639867505, 0.6472032609256847], 'activation': nn.ReLU(), 'norm': False}

In [39]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cross_val_train(config, n_splits=5):
    kf = KFold(n_splits=n_splits)
    r2_scores = []
    rmse_scores = []

    for train_index, test_index in kf.split(X_train_scaled, y_train.values):
        X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_test_cv = y_train.values[train_index], y_train.values[test_index]

        train_loader, test_loader = create_dataloader(X_train_cv, X_test_cv, y_train_cv, y_test_cv, batch_size=config["batch_size"])

        input_dim = X_train_cv.shape[1]
        output_dim = 1
        net = Net(hidden_layers=config["hidden_layers"], hidden_units=config["hidden_units"], dropout=config["dropout"], dropout_array=config["dropout_array"], input_dim=input_dim, output_dim=output_dim, activation=config["activation"], norm=config["norm"])

        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)

        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])

        for epoch in range(config["epochs"]):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            

        with torch.no_grad():
            test_loss = 0.0
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

            y_pred = net(torch.Tensor(X_test_cv).to(device)).cpu().numpy()
            r2_scores.append(r2_score(y_test_cv, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred)))
            #print(r2_scores)
            #print(rmse_scores)

    avg_r2 = np.mean(r2_scores)
    avg_rmse = np.mean(rmse_scores)

    return avg_r2, avg_rmse

best_config = {'batch_size': 8, 'lr': 0.00035262507645898283, 'epochs': 500, 'hidden_layers': 3, 'hidden_units': [128, 128, 64], 'dropout': False, 'dropout_array': [0.39233452918726697, 0.3225943639867505, 0.6472032609256847], 'activation': nn.ReLU(), 'norm': False}

#avg_r2, avg_rmse = cross_val_train(best_config)
#print(f"Average R2 score: {avg_r2}")
#print(f"Average RMSE: {avg_rmse}")


In [35]:
!pip install deap


Defaulting to user installation because normal site-packages is not writeable


In [54]:
import random
from deap import base, creator, tools, algorithms

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
fixed_config = {
    'hidden_layers': 3,
    'hidden_units': [128, 128, 64],
    'dropout': False,
    'dropout_array': [0.39233452918726697, 0.3225943639867505, 0.6472032609256847],
    'activation': nn.ReLU(),
    'norm': False
}

# Define the input variables
toolbox.register("batch_size", random.randint, 8, 100)
toolbox.register("lr", random.uniform, 1e-7, 1e-2)
toolbox.register("epochs", random.randint, 100, 500)

toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.batch_size, toolbox.lr, toolbox.epochs), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_nn(individual):
    config = {'batch_size': int(individual[0]), 'lr': np.abs(individual[1]), 'epochs': int(individual[2])}
    config.update(fixed_config)
    #print(config)

    r2, rmse = cross_val_train(config)
    #print(r2,rmse)
    return rmse

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selBest)
toolbox.register("evaluate", eval_nn)

pop = toolbox.population(n=20)
hof = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=100, stats=stats, halloffame=hof, verbose=True)

best_individual = hof[0]
print("Best individual: ", best_individual)


TypeError: object of type 'numpy.float64' has no len()

In [55]:
!pip install scikit-optimize


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 100.3/100.3 kB 5.6 MB/s eta 0:00:00


In [58]:
import numpy as np
from skopt import gp_minimize

fixed_config = {
    'hidden_layers': 3,
    'hidden_units': [128, 128, 64],
    'dropout': False,
    'dropout_array': [0.39233452918726697, 0.3225943639867505, 0.6472032609256847],
    'activation': nn.ReLU(),
    'norm': False
}

# The objective function to minimize (negative R^2)
def objective(params):
    batch_size, lr, epochs = params
    config = {'batch_size': int(batch_size), 'lr': lr, 'epochs': int(epochs)}
    config.update(fixed_config)

    r2, rmse = cross_val_train(config)
    return rmse  # Minimize negative R^2 to maximize R^2

# Hyperparameter search space
space = [
    (8, 256),  # batch_size
    (1e-4, 1e-2, 'log-uniform'),  # lr
    (100, 500)  # epochs
]

# Bayesian optimization using Gaussian Process
result = gp_minimize(
    func=objective,
    dimensions=space,
    n_calls=100,  # Number of iterations
    verbose=True
)

# Print the optimized hyperparameters
print("Best hyperparameters found:")
print("Batch size:", result.x[0])
print("Learning rate:", result.x[1])
print("Epochs:", result.x[2])


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 7.4105
Function value obtained: 816.1510
Current minimum: 816.1510
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 37.5853
Function value obtained: 756.6605
Current minimum: 756.6605
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 25.8316
Function value obtained: 750.0495
Current minimum: 750.0495
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 5.1277
Function value obtained: 4650.2176
Current minimum: 750.0495
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 12.2734
Function value obtained: 762.6817
Current minimum: 750.0495
Iteration N

C:\Users\Patrick\AppData\Roaming\Python\Python39\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 46.2285
Function value obtained: 764.0859
Current minimum: 748.4642
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 29.0385
Function value obtained: 793.5594
Current minimum: 748.4642
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 42.2569
Function value obtained: 784.2342
Current minimum: 748.4642
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 43.4520
Function value obtained: 772.5558
Current minimum: 748.4642
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 18.7825
Function value obtained: 957.8859
Current minimum: 748.4642
Iter

Iteration No: 69 ended. Search finished for the next optimal point.
Time taken: 59.3859
Function value obtained: 734.1790
Current minimum: 705.9985
Iteration No: 70 started. Searching for the next optimal point.
Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 18.6387
Function value obtained: 782.4310
Current minimum: 705.9985
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 15.5067
Function value obtained: 800.7251
Current minimum: 705.9985
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 10.2599
Function value obtained: 742.9563
Current minimum: 705.9985
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 47.1406
Function value obtained: 767.7752
Current minimum: 705.9985
Iter